In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

print(len(full_data_rows_list))

#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Task 1: Get the artist, song title, and duration of the song that was heard during sessionId = 338 and itemInSession =4

In [8]:
#First we create a relevant table to address this task. For this task we will choose itemInSession and sessionId to be the compound primary key.
query = "CREATE TABLE IF NOT EXISTS sparkify_app_history "
query = query + "(itemInSession int, sessionId int, artist varchar, firstName varchar, lastName varchar, length float, song varchar, PRIMARY KEY (itemInSession, sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sparkify_app_history (itemInSession, sessionId, artist, firstName, lastName, length, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, change the code below to `line[0], line[1]`
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[1], line[4], float(line[5]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select * from sparkify_app_history WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.length, row.song)

Faithless 495.30731201171875 Music Matters (Mark Knight Dub)


#### Get the name of the artist, song, and user(full name) for userId=10 and sessionId=182

In [11]:
## First create the table. For this task, sessionId and userId are the partition key and itemInSession is the clustering column
query = "CREATE TABLE IF NOT EXISTS sparkify_session_history "
query = query + "(sessionId int, userId int, itemInSession int, artist varchar, firstName varchar, lastName varchar, length float, song varchar, PRIMARY KEY ((sessionId, userId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sparkify_session_history ( sessionId, userId, itemInSession, artist, firstName, lastName, length, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[1], line[4], float(line[5]), line[9]))

In [ ]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select * from sparkify_session_history WHERE sessionId=182 AND userId=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.firstname,row.lastname, row.song)

#### Get every user(full name) in the music app history who listened to the song 'All Hands Against His Own'

In [ ]:
#Create the table. The primary key will be the song and the clustering columns are userId, firstName, and lastName
query = "CREATE TABLE IF NOT EXISTS sparkify_song_history "
query = query + "(song varchar, userId int, firstName varchar, lastName varchar, artist varchar, itemInSession int, sessionId int, PRIMARY KEY ((song),userId, firstName, lastName))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sparkify_song_history ( song, userId, firstName, lastName, artist, itemInSession, sessionId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4], line[0], int(line[3]), int(line[8])))

In [ ]:
query = "select firstName, lastName from sparkify_song_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname,row.lastname)

### Drop the tables before closing out the sessions

In [ ]:
##Drop the table before closing out the sessions

In [ ]:
query = "drop table sparkify_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table sparkify_session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table sparkify_song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)



### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()